# Explicaciones Contrafactuales
## Método Diverse Counterfactual Explanations (DICE)
***

* Ejemplo obtenido de la documentación de [dice-ml](https://interpret.ml/DiCE/notebooks/DiCE_getting_started.html)
* Se generan un número $k$ de instancias contrafactuales utilizando el Método presentado en [Mothilal et al., 2020]



[Mothilal et al., 2020] Mothilal, R. K., Sharma, A., and Tan, C. (2020). Explaining machine learning classifiers through diverse counterfactual explanations. In Proceedings of the 2020 Conference on fairness, accountability, and transparency, pages 607–617

In [ ]:
#!pip install dice-ml

## Bibliotecas

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

## Bibliotecca dice_ml de Python

In [ ]:
import dice_ml
from dice_ml.utils import helpers  

##  Ejemplo 1: Dataset adult_income
***
* Este dataset está disponible en [UCI ML Library](https://archive.ics.uci.edu/dataset/2/adult) y tiene asociada la tarea de clasificación

In [ ]:
dataset = helpers.load_adult_income_dataset()

In [ ]:
dataset.head()

* Este conjunto de datos tiene 8 variables, y la variable "income" tiene el valor 0 (bajo) si sus ingresos son <= 50000 o valor 1 (alto) si los ingresos son > 50000)

In [ ]:
# description of transformed features
adult_info = helpers.get_adult_data_info()
adult_info

### Generación de datos de entrenamiento y prueba

In [ ]:
target = dataset["income"]
train_dataset, test_dataset, y_train, y_test = train_test_split(dataset,
                                                                target,
                                                                test_size=0.2,
                                                                random_state=0,
                                                                stratify=target)
x_train = train_dataset.drop('income', axis=1)
x_test = test_dataset.drop('income', axis=1)

## Explicación usando el método contrafactual DICE
***
* Se genera un objeto de datos para DICE, y considerando que las variables continuas y discretas tienen diferentes formas de perturbación, se especifican los nombres de las variables continuas. 
* DICE también requiere el nombre de la variable de salida que predecirá el modelo ML.

In [ ]:
d = dice_ml.Data(dataframe=train_dataset, 
                 continuous_features=['age', 'hours_per_week'], 
                 outcome_name='income')

In [ ]:
numerical = ["age", "hours_per_week"]
categorical = x_train.columns.difference(numerical)

In [ ]:
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical)])

In [ ]:
transformations

* Con Pipeline se aplica una secuencia de transformaciones y un modelo final

In [ ]:
clf = Pipeline(steps=[('preprocessor', transformations),
                      ('classifier', RandomForestClassifier())])
model = clf.fit(x_train, y_train)



In [ ]:
test_pred = model.predict(x_test)

* DiCE tiene soporte para sklearn, tensorflow and pytorch.

In [ ]:
m = dice_ml.Model(model=model, backend="sklearn")


exp = dice_ml.Dice(d, m, method="random") # método random para la generación de contrafactuales
                                          # también permite genetic algorithm search, 
                                          # y kd-tree based generation

## Visualización de la explicación contrafactual
***
* Se buscarán dos explicaciones contrafactuales para el dato:

In [ ]:
display(x_test[0:1])

In [ ]:
print("Etiqueta del dato: ", y_test.iloc[0], "predicción del dato: ", test_pred[0])

In [ ]:
e1 = exp.generate_counterfactuals(x_test[0:1], total_CFs=2, desired_class="opposite")
e1.visualize_as_dataframe(show_only_changes=True)

In [ ]:
e1.visualize_as_dataframe(show_only_changes=False)

<div class="alert-success">
    <h2>Ejercicio</h2>
    <hr>
    <ul>
        <li> Probar explicaciones para otros datos, restringir las variables que se quieren modificar para generar los contrafactuales </li>
        <li> Revisar en la  <a href="https://interpret.ml/DiCE/notebooks/DiCE_getting_started.html">documentación</a>  como se puede generar una explicación global.</li>
    </ul>
</div>

***

## Ejemplo 2 :  Breast Cancer Dataset
***
* Este dataset está disponible en [UCI ML Library](https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic) y tiene asociada la tarea de clasificación
* Las variables se calculan a partir de una imagen digitalizada de una masa mamaria, y  describen características de los núcleos celulares presentes en la imagen. 

In [ ]:
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

## Descarga de datos

In [ ]:
breast_cancer = load_breast_cancer()

breast_cancer_df = pd.DataFrame(data=breast_cancer.data, columns=breast_cancer.feature_names)
breast_cancer_df["TumorType"] = breast_cancer.target

breast_cancer_df.head()

## Conjuntos de entrenamiento y prueba

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(breast_cancer.data, 
                                                    breast_cancer.target,
                                                    train_size=0.80,
                                                    random_state=0)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
model = Sequential([
            Dense(50, activation="relu", input_shape=(len(breast_cancer.feature_names), )),
            Dense(50, activation="relu"),
            Dense(50, activation="relu"),
            Dense(1, activation="sigmoid"),
           ])

model.summary()

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["Precision"])

In [ ]:
%%time

history = model.fit(X_train, y_train, batch_size=8, epochs=10, verbose=0)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

test_pred = [0 if pred< 0.5 else 1 for pred in model.predict(X_test).flatten()]
train_pred = [0 if pred< 0.5 else 1 for pred in model.predict(X_train).flatten()]

print("Train Accuracy : %.2f"%accuracy_score(y_train, train_pred))
print("Test  Accuracy : %.2f"%accuracy_score(y_test, test_pred))

print("\nTest  Classification Report : ")
print(classification_report(y_test, test_pred))

In [ ]:
d = dice_ml.Data(dataframe=breast_cancer_df,
                 continuous_features=breast_cancer.feature_names.tolist(),
                 outcome_name='TumorType')

m = dice_ml.Model(model=model, backend="TF2")

exp = dice_ml.Dice(d, m)



In [ ]:
breast_cancer_df.head()

In [ ]:
X_train = pd.DataFrame(X_train, columns = breast_cancer_df.columns[:-1])
X_test = pd.DataFrame(X_test, columns=breast_cancer_df.columns[:-1])

In [ ]:
dice_exp = exp.generate_counterfactuals(X_test[0:1], total_CFs=3,desired_class="opposite")

In [ ]:
dice_exp.visualize_as_dataframe()

In [ ]:
dice_exp.visualize_as_dataframe(show_only_changes=True)